This is a step-by-step simplified implementation of a Retrieval-Augmented Generation (RAG) pipeline for business analysis document querying.

In the first step, I set up the configuration for the RAG system, specifying the PDF document (BABOK Guide v3) and the language model (TinyLlama) to be used.

In [53]:
PDF_FILE = r"BABOK-Guide-v3-Member.pdf"
MODEL = "tinyllama"

All necessary dependencies are installed, including:

LangChain: Core framework for building LLM application

PyPDF: For PDF document processng

FAISS: Vector database for similarity sarch

Sentence Transformers: For creating text embddings

LangChain Ollama: Integration with Ollama models

In [55]:
!pip install langchain langchain-community pypdf faiss-cpu sentence-transformers langchain-ollama

Defaulting to user installation because normal site-packages is not writeable


Next, I use PyPDFLoader to extract text from the 514-page BABOK PDF document, converting each page into a document object with metadata.

In [57]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content)

Number of pages: 514
Length of a page: 0
Content of a page: 


The large document is broken down into smaller, manageable chunks of 1,500 characters with 100-character overlap.This creates 949 chunks that are optimal for embedding and retrieval.

In [59]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)

Number of chunks: 949
Length of a chunk: 62
Content of a chunk: BABOK
®
v3
A GUIDE TO THE BUSINESS ANALYSIS
BODY OF KNOWLEDGE®


Creates numerical embeddings for each text chunk using the MiniLM sentence transformer model, then stores them in a FAISS vector database for efficient similarity search.

In [61]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)

C:\Users\GoreMethi\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


To test the retrieval system, I search for relevant document chunks about "requirement elicitation" and expect it to return the top matching results with metadata.

In [63]:
retriever = vectorstore.as_retriever()
retriever.invoke("What is requirement elicitation?")

C:\Users\GoreMethi\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(id='11d85d5b-6385-4b7d-94f0-6fde8ced36cc', metadata={'producer': 'Acrobat Distiller 11.0 (Windows)', 'creator': 'FrameMaker 12.0.2', 'creationdate': '2015-11-26T10:29:02+00:00', 'author': 'IIBA', 'moddate': '2020-03-27T14:56:48+07:00', 'title': 'BABOK Guide v3', 'webstatement': 'www.iiba.org', 'source': 'BABOK-Guide-v3-Member.pdf', 'total_pages': 514, 'page': 70, 'page_label': '71'}, page_content='from people or documents—because it is unknown. Experiments can help \ndiscover this kind of information. Experiments include observational studies, \nproofs of concept, and prototypes.\nOne or more elicitation techniques may be used to produce the desired outcome \nwithin the scope of elicitation.\nStakeholders may collaborate in elicitation by:\n• participating and interacting during the elicitation activity, and\n• researching, studying, and providing feedback on documents, systems, \nmodels, and interfaces.\n4.2.3 Inputs\n• Elicitation Activity Plan: includes the planned elicita

Initialized the TinyLlama language model through Ollama with zero temperature to fetch deterministic responses.

In [67]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=MODEL, temperature=0)
model.invoke("Create a list of important stakeholders to consider in requirement gathering")

AIMessage(content="1. Product Owner: The product owner is the primary stakeholder involved in requirements gathering. They are responsible for defining the requirements and ensuring that they align with the business goals and objectives.\n\n2. Business Analysts (BAs): BAs are responsible for translating the product owner's requirements into technical specifications, which are then used to create a detailed requirement specification document.\n\n3. Designers: Designers are responsible for creating user-friendly interfaces and visual designs that align with the requirements. They also help in creating wireframes and prototypes.\n\n4. Developers: Developers are responsible for implementing the requirements into software code. They work closely with designers to ensure that the software meets the requirements.\n\n5. Testers: Testers are responsible for testing the software to ensure that it meets the requirements. They also help in identifying and fixing any bugs or issues that arise durin

Sets up a simple chain that processes model output into clean string format for better readability and tests the chain with a sample question about stakeholders

In [69]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("Create a list of important stakeholders to consider in requirement gathering"))

1. Product Owner: The product owner is the primary stakeholder involved in requirements gathering. They are responsible for defining the requirements and ensuring that they align with the business goals and objectives.

2. Business Analysts: Business analysts are responsible for gathering requirements from various departments within the organization. They work closely with product owners to understand the business needs and translate them into requirements.

3. User Experts: User experts are involved in testing and validating the requirements. They test the requirements against real-life scenarios and provide feedback on how they can be improved.

4. Designers: Designers work with product owners to create user interfaces, wireframes, and prototypes that demonstrate the functionality of the system.

5. Developers: Developers are responsible for creating the software components required to implement the requirements. They work closely with designers to ensure that the software meets the 

Created a structured prompt template that instructs the model to act as a business analyst assistant and only answer based on provided context.

In [71]:
from langchain.prompts import PromptTemplate

template = """
You are a business analyst assistant that provides answers to questions based on
the information given via context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


You are a business analyst assistant that provides answers to questions based on
the information given via context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: Here is some context

Question: Here is a question



Tests the prompt template with a simple example to verify the chain works correctly.
However, as you can see the test fails due to several below possible reasons, 
1. Model limitations (TinyLlama is very small)
tinyllama is only 1.1B parameters, which is very tiny by LLM standards. It has limited reasoning ability, so it may not understand relational logic like:
“Pankaj is married to Pallavi, then Pallavi is married to Pankaj”

2. The context isn't reinforced well in the prompt
Without a carefully worded prompt, small models struggle to "bind" context and question together.

3. No RAG behavior is involved yet
This test doesn’t use retrieval. It’s just testing if the model can reason with a short given context, which is more like a closed-book quiz for a very small model

In [75]:
chain = prompt | model | parser

chain.invoke({
    "context": "Pankaj is married to Pallavi", 
    "question": "Who is Pallavi married to?"
})

'Answer: Pankaj is not married.'

This is the core RAG pipeline that:

Takes a question as input
Uses the retriever to find relevant document chunks (context)
Combines the context and question in the prompt template
Sends to the language model for generation
Parses the output into a clean string

In [77]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

Tests the complete RAG system with a business analysis question about "Business Planning and Monitoring phase challenges" and demonstrates the full workflow from question to answer.

In [79]:
questions = [
    "What are main challenges in Business Planning and Monitoring phase?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")
     

Question: What are main challenges in Business Planning and Monitoring phase?


C:\Users\GoreMethi\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer: The main challenges in the Business Planning and Monitoring phase include:
1. Identifying gap(s) and needs: The business analyist must identify gaps or needs that exist within the current state of a process, function, or system. This requires understanding the current situation and identifying areas where improvements can be made.
2. Defining alternative solutions: Once gaps are identified, the business analyist must define alternative solutions to address them. This involves considering various options and selecting the most appropriate one based on their feasibility, cost-effectiveness, and other relevant factors.
3. Evaluating alternatives: The business analyist evaluates the alternatives by comparing their benefits and drawbacks. They also consider the potential impact of each alternative on stakeholders and the wider organization.
4. Selecting a solution: Once all alternatives have been evaluated, the business analyist selects the most appropriate solution based on its fea